# LSTM-based model for time series data

This notebook provides tasks to analyze the flu-trends dataset using a Long Short-Term Memory (LSTM) model.

Required Python packages: pandas, numpy, matplotlib, darts, optuna

## Task 1: Data Preparation


- Load the dataset and split in into train/test/validation the same way you did yesterday
- Convert the dataset into darts' internal format and plot the data.
- Split the dataset into training and test sets and plot the curves.

## Task 2: Recap - XGBoost Model

To familiarize yourself with the new library, you should first look at its documentation. Then, train a XGBoost model on the data set. Just like yesterday. You can either use the default parameters, or you can use the ones you think work best. Compute the same metrics and compare the results.

In [ ]:
from darts.models import RandomForest

...

## Task 3: Training an LSTM-based model

We will begin to work with neural networks for time series prediction. One type of neural network is a recurrent neural network (RNN). One of the most popular RNNs is the Long Short-Term Memory (LSTM) network.

Your task is to

- Use the RNN model from the Darts library.
- Train the model on the training set
- Make predictions on the test set
- Compute the same metrics as in the previous task and compare the results.

Unlike tree-based models, neural networks are more sensitive to the scale of the input data. Therefore, you should normalize the data before training the model. For example, you can use a min-max scaler that scales the data to the range [0, 1].

## Task 4: Optuna-based hyperparameter optimization with time series cross-validation

Today you will use Optuna to optimize the hyperparameters of the LSTM model. Compared to a grid search-based approach, Optuna is more efficient because it uses a more sophisticated algorithm to search the hyperparameter space. You will also use time series cross-validation to ensure that the model is evaluated on a realistic scenario.

Before you start working on this exercise, you should familiarize yourself with [Optuna](https://optuna.org/#key_features). Then you can take a look at the [darts documentation](https://unit8co.github.io/darts/userguide/hyperparameter_optimization.html#hyperparameter-optimization-with-optuna) to see how to use Optuna with darts.

While it is not mandatory, we would like you to try to implement a class called RnnOptimization, which will handle the optimization and final training of the LSTM model. This class should have at least four methods:

- __init__(self, ...)
- objective(self, trial)
- optimize(self)
- train(self, best_params)

Note: Save your optuna study as a sqlite database to avoid losing the results.

## Task 5. Visualizing the Results of Hyperparameter Optimization

To understand the optimization process, you should visualize the results. You should create the following plots:

- A plot showing the trials and their results
- A plot showing the distribution of the results for each hyperparameter
- A plot showing the importance of each hyperparameter
- A plot showing a 3D surface plot of the two most important hyperparameters

## Task 6: Final Evaluation

Evaluate your model using the test data. Compute metrics (such as RMSE or MAPE) to compare the performance of your LSTM model with the RF and/or XGBoost models. Feel free to repeat the training and evaluation of the ARIMA, Random Forest, and XGBoost models if you want to change the optimization part or have a more consistent approach. Otherwise, you can use the results from the previous days. Visualize the results and discuss them.

## Optional tasks

Feel free to try anything that you find interesting (ask us for help if you need it). Otherwise, here are some suggestions:

- Try adding different features. Systematically analyze the impact of the additional feature on the model performance. Have a look at systematic feature selection: https://scikit-learn.org/stable/modules/feature_selection.html
- Have a look at the [Backtesting](https://unit8co.github.io/darts/quickstart/00-quickstart.html#Backtesting:-simulate-historical-forecasting) section of the documentation. Perform an error and residual analysis for one of the models and discuss the results.
- If you did not do it yet, try to optimize the number of lags used in the model. Treat it as a hyperparameter.
- Does it make a difference if you use the GRU model instead of the LSTM model? Change the parameters accordingly and compare the results.